# Developing NDK to JSON

Using specifications [here](https://www.ldeo.columbia.edu/~gcmt/projects/CMT/catalog/allorder.ndk_explained)

In [1]:
#import SeismoTools as st
import json

In [2]:
!ls data/GCMT/

aug14.ndk.txt


In [3]:
filename = './data/GCMT/aug14.ndk.txt'

In [4]:
with open(filename, mode='r') as f:
    filecontent = f.read()

In [5]:
filelines = filecontent.splitlines()

In [6]:
n_events = len(filelines) // 5
n_events

185

In [7]:
raw_events = [filelines[(i*5):((i*5)+5)] for i in range(n_events)]
raw_events

[['PDEW 2014/08/01 00:01:27.8 -57.40 -147.53  10.0 0.0 4.8 PACIFIC-ANTARCTIC RIDGE ',
  'C201408010001B   B: 27   29  40 S: 71   86  50 M:  0    0   0 CMT: 1 TRIHD:  0.7',
  'CENTROID:      5.7 0.3 -57.71 0.03 -147.76 0.03  20.4  2.2 FREE S-20141113124318',
  '23  0.008 0.159  1.870 0.145 -1.880 0.118 -0.469 0.286 -0.335 0.213  1.810 0.108',
  'V10   2.717 12 158  -0.101 78 353  -2.618  3 248   2.668 294 80    6 202 84  170'],
 ['PDEW 2014/08/01 03:10:07.2   4.53   96.48  34.5 0.0 4.8 NORTHERN SUMATRA, INDONE',
  'C201408010310A   B: 32   34  40 S: 98  151  50 M:  0    0   0 CMT: 1 TRIHD:  0.7',
  'CENTROID:      1.9 0.3   4.48 0.02   96.55 0.01  20.7  1.7 FREE S-20141105143301',
  '23  0.002 0.142 -3.100 0.115  3.100 0.117 -1.460 0.255 -0.817 0.230 -0.737 0.092',
  'V10   3.326 12  86   0.497 64 203  -3.821 22 351   3.573 130 65 -172  37 83  -25'],
 ['PDEW 2014/08/01 04:11:16.5  36.86    3.18  10.0 0.0 5.5 NORTHERN ALGERIA        ',
  'C201408010411A   B:120  199  40 S:155  332  50 M:

In [8]:
raw_events[4]

['PDEW 2014/08/01 13:12:56.1  17.51  146.53 108.7 0.0 5.0 MARIANA ISLANDS         ',
 'C201408011312A   B: 52   57  40 S:120  189  50 M:  0    0   0 CMT: 1 TRIHD:  0.8',
 'CENTROID:      3.0 0.2  17.53 0.01  146.79 0.02 101.9  2.7 FREE S-20141105154027',
 '23  0.507 0.116  1.300 0.116 -1.810 0.111 -2.800 0.064  0.600 0.074  2.610 0.093',
 'V10   4.205 33 160  -0.017 46 293  -4.191 25  52   4.198 193 47  173 288 85   44']

In [ ]:
def NDK2JSON(filename_in, i, filename_out):
    
    filelines = get_NDK_lines(filename_in)
    n_events = len(filelines) // 5
    raw_events = [filelines[(j*5):((j*5)+5)] for j in range(n_events)]
    data = parse_lines(raw_events[i])
    
    with open(filename_out, 'w') as f:
        json.dump(data, f, indent=4)
    return data
        
def get_NDK_lines(filename_in):
    with open(filename_in, mode='r') as f:
        filecontent = f.read()
    filelines = filecontent.splitlines()
    return filelines

def parse_lines(raw_lines):
    data = {
        **parse_line1(raw_lines[0]),
        **parse_line2(raw_lines[1]),
        **parse_line3(raw_lines[2]),
        **parse_line4(raw_lines[3]),
        **parse_line5(raw_lines[4]),
    }
    print(json.dumps(parse_line3(raw_lines[2]), indent=2, default=str))
    return data

def irange(istart, iend):
    return slice(istart-1,iend)

# Lines parsing
def parse_line1(raw_line):
    data = {
        'hypocenter_reference_catalog' : raw_line[irange(1,4)],
        'date' : raw_line[irange(6,15)],
        'time' : raw_line[irange(17,26)],
        'latitude' : raw_line[irange(28,33)],
        'longitude' : raw_line[irange(35,41)],
        'depth' : raw_line[irange(43,47)],
        'magnitudes' : raw_line[irange(49,55)],
        'location' : raw_line[irange(57,80)],
    }
    data = {k: v.strip() for k, v in data.items()}
    return data
def parse_line2(raw_line):
    data = {
        'CMT_event_name' : raw_line[irange(1,16)].strip(),
        'inversion_info' : parse_inversion_info(raw_line[irange(18,61)]),
        'inversion_source_type' : raw_line[irange(63,68)].strip(),
        'moment_rate_function' : parse_inversion_source_type(raw_line[irange(70,80)]),
    }
    return data
def parse_line3(raw_line):
    data = {
        'centroid_parameters' : parse_centroid(raw_line[irange(1,58)]),
        'depth_type' : parse_depth_type(raw_line[irange(60,63)]),
        'timestamp' : parse_timestamp(raw_line[irange(65,80)]),
    }
    return data
def parse_line4(raw_line):
    data = {
        'CMT_exponent' : raw_line[irange(1,2)],
        'moment_tensor' : raw_line[irange(3,80)],
    }
    return data
def parse_line5(raw_line):
    data = {
        'code_version' : raw_line[irange(1,3)],
        'moment_tensor_principal_axis' : raw_line[irange(4,48)],
        'scalar_moment' : raw_line[irange(50,56)],
        'sdr_1st_nodal_plane_dc' : raw_line[irange(58,80)],
    }
    return data

# Line 2 parsing
def parse_inversion_info(raw_line_string):
    data = {
        'B_data' : parse_inversion_info_B(raw_line_string),
        'S_data' : parse_inversion_info_S(raw_line_string),
        'M_data' : parse_inversion_info_M(raw_line_string),
    }
    return data
def parse_inversion_info_B(raw_line_string):
    if 'B:' in raw_line_string:
        istart = raw_line_string.find('B:') + 2
        iend = raw_line_string.find('S')
        B_string = raw_line_string[istart:iend]
        B_string_split = B_string.split()
        n_stations = B_string_split[0]
        n_components = B_string_split[1]
        shortest_period = B_string_split[2]
    else:
        n_stations = ''
        n_components = ''
        shortest_period = ''
    data = {
        'n_stations' : n_stations,
        'n_components' : n_components,
        'shortest_period' : shortest_period,
    }
    return data
def parse_inversion_info_S(raw_line_string):
    if 'S:' in raw_line_string:
        istart = raw_line_string.find('S:') + 2
        iend = raw_line_string.find('M')
        S_string = raw_line_string[istart:iend]
        S_string_split = S_string.split()
        n_stations = S_string_split[0]
        n_components = S_string_split[1]
        shortest_period = S_string_split[2]
    else:
        n_stations = ''
        n_components = ''
        shortest_period = ''
    data = {
        'n_stations' : n_stations,
        'n_components' : n_components,
        'shortest_period' : shortest_period,
    }
    return data
def parse_inversion_info_M(raw_line_string):
    if 'M:' in raw_line_string:
        istart = raw_line_string.find('M:') + 2
        iend = len(raw_line_string)
        M_string = raw_line_string[istart:iend]
        M_string_split = M_string.split()
        n_stations = M_string_split[0]
        n_components = M_string_split[1]
        shortest_period = M_string_split[2]
    else:
        n_stations = ''
        n_components = ''
        shortest_period = ''
    data = {
        'n_stations' : n_stations,
        'n_components' : n_components,
        'shortest_period' : shortest_period,
    }
    return data
def parse_inversion_source_type(raw_line_string):
    colon_loc = raw_line_string.find(':')
    type_string = raw_line_string[0:colon_loc]
    value_string = raw_line_string[(colon_loc+1):len(raw_line_string)]
    data = {
        'type' : type_string,
        'value' : value_string,
    }
    data = {k: v.strip() for k, v in data.items()}
    return data

# Line 3 parsing
def parse_centroid(raw_line_string):
    raw_line_string_split = raw_line_string.split()
    centroid_time = {
        'value' : raw_line_string_split[1],
        'standard_error' : raw_line_string_split[2],
    }
    centroid_latitude = {
        'value' : raw_line_string_split[3],
        'standard_error' : raw_line_string_split[4],
    }
    centroid_longitude = {
        'value' : raw_line_string_split[5],
        'standard_error' : raw_line_string_split[6],
    }
    centroid_depth = {
        'value' : raw_line_string_split[7],
        'standard_error' : raw_line_string_split[8],
    }
    data = {
        'time' : centroid_time,
        'latitude' : centroid_latitude,
        'longitude' : centroid_longitude,
        'depth' : centroid_depth,
    }
    return data
def parse_depth_type(raw_line_string):
    if 'FREE' in raw_line_string:
        depth_type = 'free'
    elif 'FIX' in raw_line_string:
        depth_type = 'fixed'
    elif 'BDY' in raw_line_string:
        depth_type = 'modelled'
    else: 
        depth_type = 'unknown'
    return depth_type
def parse_timestamp(raw_line_string):
    if 'Q-' in raw_line_string:
        CMT_type = 'quick'
    elif 'S-' in raw_line_string:
        CMT_type = 'standard'
    else: 
        CMT_type = 'unknown'
    data = {
        'CMT_type' : CMT_type,
        'Timestamp' : raw_line_string.strip(),
    }
    return data
        
parse_lines(raw_events[4])

In [ ]:
NDK2JSON('./data/GCMT/aug14.ndk.txt', 4, 'C201408011312A.json')

In [ ]:
!ls

# Using package

In [1]:
import SeismoTools as st

In [2]:
filename = './data/GCMT/aug14.ndk.txt'

In [3]:
!ls data/GCMT/

aug14.ndk.txt


In [4]:
!ls

README.md                    test
SeismoTools                  xl-data.ipynb
data                         xl.json
developing-NDK-to-JSON.ipynb xl.tar.gz
developing-SAC-to-JSON.ipynb


In [5]:
!rm C201408011312A.json

rm: C201408011312A.json: No such file or directory


In [6]:
st.NDK2JSON(filename, 4, 'C201408011312A.json')

{'hypocenter_reference_catalog': 'PDEW',
 'date': '2014/08/01',
 'time': '13:12:56.1',
 'latitude': '17.51',
 'longitude': '146.53',
 'depth': '108.7',
 'magnitudes': '0.0 5.0',
 'location': 'MARIANA ISLANDS',
 'CMT_event_name': 'C201408011312A',
 'inversion_info': {'B_data': {'n_stations': '52',
   'n_components': '57',
   'shortest_period': '40'},
  'S_data': {'n_stations': '120',
   'n_components': '189',
   'shortest_period': '50'},
  'M_data': {'n_stations': '0', 'n_components': '0', 'shortest_period': '0'}},
 'inversion_source_type': 'CMT: 1',
 'moment_rate_function': {'type': 'TRIHD', 'value': '0.8'},
 'centroid_parameters': {'time': {'value': '3.0', 'standard_error': '0.2'},
  'latitude': {'value': '17.53', 'standard_error': '0.01'},
  'longitude': {'value': '146.79', 'standard_error': '0.02'},
  'depth': {'value': '101.9', 'standard_error': '2.7'}},
 'depth_type': 'free',
 'timestamp': {'CMT_type': 'standard', 'Timestamp': 'S-20141105154027'},
 'CMT_exponent': '23',
 'moment_t

In [7]:
st.NDK2JSON_list(filename, range(185),'ndk_2014_08_01.json')

[{'hypocenter_reference_catalog': 'PDEW',
  'date': '2014/08/01',
  'time': '00:01:27.8',
  'latitude': '-57.40',
  'longitude': '-147.53',
  'depth': '10.0',
  'magnitudes': '0.0 4.8',
  'location': 'PACIFIC-ANTARCTIC RIDGE',
  'CMT_event_name': 'C201408010001B',
  'inversion_info': {'B_data': {'n_stations': '27',
    'n_components': '29',
    'shortest_period': '40'},
   'S_data': {'n_stations': '71',
    'n_components': '86',
    'shortest_period': '50'},
   'M_data': {'n_stations': '0', 'n_components': '0', 'shortest_period': '0'}},
  'inversion_source_type': 'CMT: 1',
  'moment_rate_function': {'type': 'TRIHD', 'value': '0.7'},
  'centroid_parameters': {'time': {'value': '5.7', 'standard_error': '0.3'},
   'latitude': {'value': '-57.71', 'standard_error': '0.03'},
   'longitude': {'value': '-147.76', 'standard_error': '0.03'},
   'depth': {'value': '20.4', 'standard_error': '2.2'}},
  'depth_type': 'free',
  'timestamp': {'CMT_type': 'standard', 'Timestamp': 'S-20141113124318'},
 

In [8]:
!ls

C201408011312A.json          ndk_2014_08_01.json
README.md                    test
SeismoTools                  xl-data.ipynb
data                         xl.json
developing-NDK-to-JSON.ipynb xl.tar.gz
developing-SAC-to-JSON.ipynb
